<a href="https://colab.research.google.com/github/JacksonLeb/tensorflow_learning/blob/main/08_rnn_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#introduction to NLP fundamentals in Tf

##Getting info from natural language (sequences or speech)
##NLP can also be called seq2seq problems

###Checking for GPU


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9b828122-bd48-c505-460a-ad4a27a6c71e)


In [2]:
##Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-27 22:40:53--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-27 22:40:53 (80.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
##Get a text dataset
###Using the Kaggle Intro to NLP dataset, samples of tweets labeled hazard or not
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-07-27 22:40:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-07-27 22:40:58 (102 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

##Visualizing a text dataset

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [9]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [10]:
#shuffle trianing df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
#test df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
#examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
#total num of samples
len(train_df), len(test_df)

(7613, 3263)

In [14]:
#visualizing random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random index
for row in train_df_shuffled[["text", "target"]][random_index:random_index +5 ].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}")

Target: 0 (not real disaster)
Text:
The Civil Rights Movement (Eyewitness History Series) Wexler Sanford Hardcover http://t.co/kK0qFKGqcY http://t.co/SMc1Ro09Fs
Target: 0 (not real disaster)
Text:
One of these candidates is going to have a Frank Grimes level meltdown with how voters love Trump no matter what. http://t.co/pBEgOf4740
Target: 0 (not real disaster)
Text:
BOOM! Your country was just entirely annihilated by a hÛ_ ÛÓ Britain https://t.co/IrFCn71sZv
Target: 0 (not real disaster)
Text:
Please sign &amp; RT to save #SaltRiverWildHorses http://t.co/IKUAYUSEqt http://t.co/BQBHUyfmE9
Target: 0 (not real disaster)
Text:
'Nobody remembers who came in second.' Charles Schulz


In [15]:
#splittting data into training and validation
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=30)

In [16]:
#checking lengths of each data set
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [17]:
#check the first 10 sentences
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [18]:
val_sentences[:10]

array(['Blaze is my bro http://t.co/UdKeSJ01mL',
       'I was taught at school in the 1970s that piracy slavery and suicide-bombing were purely historical. No one then expected them to re-occur',
       'http://t.co/FueRk0gWui Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/Mv7GgGlmVc',
       "I'm a friggin wreck destiel sucks (read the vine description)\nhttps://t.co/MKX6Ux4OZt",
       'Dr. Bengston on #wildfire management: \x89ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.\x89Û\x9d #smem',
       '@ameenshaikh3 sir i just only wanted to make a point about @sureshpprabhu you made and said he is lying about bridge collapse.',
       'Wow! I just won this for free The Hobbit: Desolation of Smaug UV digital download code *GIN 9 http://t.co/MjFdCrjs8j #listia',
       'Photoset: hakogaku: ?åÊI am a kurd. i was born on a battlefield. raised on a battlefield. gunfire sirens... http://t.co/obp595W7tm',
       'Wash

## Converting text into numbers

#Tokenization and Embedding
##Tokenization: Each word/character is set to a number/one-hot encoding
##Embedding: Each word/character is set to a vector, with user-defined size, allowing for information on the relationship between tokens/words/characters. Additionally, can limit size and can be learned (treated as a weight matrix)


In [19]:
#Text Vectorization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

#Use default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long each batch of a setence should be
                                    pad_to_max_tokens=False)


In [20]:
#find average number of tokens (words) in the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
#set up text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocabulary
max_length = 15 #max length of a tweet that our models sees

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=max_length, #how long each batch of a setence should be
                                    pad_to_max_tokens=True)

In [22]:
#adapting text_vectorizer to training data
text_vectorizer.adapt(train_sentences) #adapt to training data, very important

In [23]:
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [24]:
#Create sample setnces and tokenize it
sample_sentence = "A tornado destroyed my house"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  3, 595, 346,  13, 311,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
#Choose a random sentence from the triaing dataset and vectorize it
random_sentence = random.choice(train_sentences)
print(f"original sentence: {random_sentence}\n vectorized sentence: {text_vectorizer([random_sentence])}")

original sentence: #NoChillLukeHammings
IM SCREAMING
 vectorized sentence: [[  1  32 301   0   0   0   0   0   0   0   0   0   0   0   0]]


In [26]:
#number of unique words found in train_sentences
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Num of words: {len(words_in_vocab)}")
top_5_words, bottom_5_words

Num of words: 10000


(['', '[UNK]', 'the', 'a', 'in'],
 ['panda', 'panama', 'pampalmater', 'palmoil', 'palmer'])

### Creating an embedding using an embedding layer


In [27]:
#using tensorflow.keras.layeres.Embedding
##Parameters: input_dim, output_dim, input_length
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [28]:
#get a random sentence
random_sentence = random.choice(train_sentences)
print(f"random sentence: {random_sentence}")
print(f"embedded:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

random sentence: Cross-border terrorism: Pakistan caught red-handed again http://t.co/uDj50J3MV4
embedded:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03958807, -0.01525826,  0.00317426, ...,  0.02307819,
         -0.02895102,  0.04406451],
        [-0.04168677,  0.04757056, -0.04176748, ...,  0.0343872 ,
          0.00117552, -0.04675726],
        [-0.0404402 , -0.01409991, -0.01986741, ..., -0.03874302,
          0.03078927,  0.04022708],
        ...,
        [ 0.03095423,  0.02592571,  0.02956278, ..., -0.00507424,
         -0.00331098, -0.01307736],
        [ 0.03095423,  0.02592571,  0.02956278, ..., -0.00507424,
         -0.00331098, -0.01307736],
        [ 0.03095423,  0.02592571,  0.02956278, ..., -0.00507424,
         -0.00331098, -0.01307736]]], dtype=float32)>

In [29]:
sample_embed[0][0], sample_embed[0][0].shape, 

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03958807, -0.01525826,  0.00317426,  0.04912117,  0.04147414,
         0.04636774, -0.02450562,  0.00150914, -0.04032522,  0.00096303,
        -0.01658206, -0.01557084, -0.03651081,  0.02148367, -0.03764522,
        -0.00326121, -0.00012162,  0.02221525, -0.02461655, -0.03417462,
         0.0475762 , -0.01146208, -0.02497067, -0.000204  ,  0.03244803,
        -0.02410991,  0.02290993,  0.04728557,  0.01609837,  0.02175493,
         0.01939486,  0.01252573,  0.01923558,  0.04589095,  0.04481547,
        -0.00564826,  0.03999725, -0.02430788,  0.03627784,  0.02058608,
        -0.03298042,  0.01547693,  0.00061299,  0.03878326,  0.02189846,
         0.01030647,  0.00094247,  0.03160058, -0.02842424,  0.01844959,
         0.02329219,  0.01505197,  0.01270675,  0.02295461, -0.0068306 ,
        -0.00636365,  0.03253921,  0.00347856, -0.03884237, -0.01305293,
         0.04648313, -0.03485196, -0.04880828, -0.00329009, -0.00011702,
  

##Modelling a text dataset (running series of experiments)
###1. Naive bayes with Tf-IDF encoder (BASELINE)
###2. Feed-forward neural network 
###3. LSTM
###4. GRU
###5. Bidirectiona;-LSTM
###6. 1D CNN
###7. TFHub pretrianed feature extractor
###8. TFHub pretrained with 10% of trained datam



In [30]:
###Model 0: Getting a baseline - Skleanrs multinomial naive bayes with tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #convert words to numbers
                    ("clf", MultinomialNB()) #model the text

])

#fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
#Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)

In [32]:
baseline_score

0.8044619422572179

In [33]:
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 80.45%


In [34]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:30]

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1])

In [35]:
### creating an evaluation function for our modeling experiements
#Accuracy, Precision, Recall and f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, preicision, recall and f1-score
  """
  #calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  
  #precision, recall and f1-score(using weighted average)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [36]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 80.4461942257218,
 'f1': 0.7993754401853582,
 'precision': 0.8113250534023555,
 'recall': 0.8044619422572179}

In [37]:
### Model 1: Feed Forward NN

# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"


In [38]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [41]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20220727-224108
Epoch 1/5
215/215 [==============================] - 8s 7ms/step - loss: 0.6114 - accuracy: 0.7006 - val_loss: 0.5164 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4426 - accuracy: 0.8164 - val_loss: 0.4430 - val_accuracy: 0.7992
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3483 - accuracy: 0.8606 - val_loss: 0.4309 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2852 - accuracy: 0.8899 - val_loss: 0.4399 - val_accuracy: 0.8189
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2372 - accuracy: 0.9107 - val_loss: 0.4576 - val_accuracy: 0.7992


In [42]:
#check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4576 - accuracy: 0.7992


[0.4576182961463928, 0.7992125749588013]

In [43]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[3.65742035e-02],
       [5.76329708e-01],
       [9.97434318e-01],
       [9.18445438e-02],
       [9.75460947e-01],
       [3.89407575e-02],
       [7.12442677e-03],
       [1.64515719e-01],
       [9.96310294e-01],
       [9.80351150e-01],
       [2.03029960e-02],
       [1.39970303e-01],
       [9.99522448e-01],
       [5.71614802e-02],
       [3.50430794e-02],
       [4.50059950e-01],
       [9.97578204e-01],
       [1.88367814e-01],
       [4.99655366e-01],
       [3.41268063e-01],
       [8.13926905e-02],
       [2.62851715e-01],
       [8.97087216e-01],
       [4.90057647e-01],
       [2.95827806e-01],
       [9.98772800e-01],
       [1.76873162e-01],
       [1.87893845e-02],
       [9.62860346e-01],
       [9.99876499e-01],
       [9.99330521e-01],
       [1.49382338e-01],
       [3.18580121e-01],
       [3.29108909e-02],
       [5.14534339e-02],
       [9.57416534e-01],
       [5.55360783e-03],
       [3.48500073e-01],
       [3.05935860e-01],
       [3.67221422e-02],


In [44]:
model_1_pred_probs[0]

array([0.0365742], dtype=float32)

In [45]:
model_1_pred_probs.shape

(762, 1)

In [46]:
model_1_pred_probs[:10]

array([[0.0365742 ],
       [0.5763297 ],
       [0.9974343 ],
       [0.09184454],
       [0.97546095],
       [0.03894076],
       [0.00712443],
       [0.16451572],
       [0.9963103 ],
       [0.98035115]], dtype=float32)

In [47]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1

In [48]:
#Calculate model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.92125984251969,
 'f1': 0.7969201794869142,
 'precision': 0.7994614695308752,
 'recall': 0.7992125984251969}

In [49]:
baseline_results = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [50]:
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'me',
 'as',
 'up',
 'just',
 'but',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'its',
 'all',
 'will',
 'after',
 'no',
 'fire',
 'has',
 'an',
 'when',
 'if',
 'get',
 'now',
 'we',
 'about',
 'new',
 'via',
 'more',
 'or',
 'people',
 'what',
 'news',
 'one',
 'how',
 'he',
 'they',
 'over',
 'dont',
 'been',
 'who',
 'into',
 'video',
 'were',
 'do',
 'can',
 '2',
 'us',
 'disaster',
 'emergency',
 'there',
 'police',
 'than',
 'would',
 'still',
 'his',
 'some',
 'her',
 'body',
 'burning',
 'storm',
 'crash',
 'back',
 'had',
 'suicide',
 'man',
 'why',
 'know',
 'got',
 'them',
 'buildings',
 'world',
 'time',
 'off',
 'see',
 'california',
 'first',
 'day',
 'rt',
 'bomb',
 'youtube',
 'cant',
 'love',
 'nuclear',
 'going',
 'their',
 'dead',
 'two',
 'train',
 'killed',
 'go'

In [51]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [52]:
#get weight matrix of embedding layer
#numerical representations of each token in our training data, learned for 5 epochs
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [53]:
embed_weights

array([[ 0.05262721,  0.04759351,  0.01038965, ...,  0.00893207,
        -0.02381207, -0.03386072],
       [-0.00069337,  0.04993873, -0.0437829 , ..., -0.00193313,
         0.02259258,  0.00392428],
       [ 0.02504043,  0.04856981, -0.07306533, ...,  0.0372062 ,
        -0.04597528, -0.06454707],
       ...,
       [-0.07607889, -0.05238977,  0.01256561, ..., -0.01529595,
         0.04627615,  0.10393449],
       [ 0.00108855, -0.03103786, -0.00651623, ..., -0.05128421,
         0.07825278, -0.00570883],
       [ 0.01533067,  0.06531977, -0.0465888 , ...,  0.08966628,
        -0.02825991, -0.09583197]], dtype=float32)

In [54]:
#projector tool
import io
out_v = io.open('vectors.tsv', 'w', encoding="utf-8")
out_m = io.open('metadata.tsv', 'w', encoding="utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue

  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [55]:
#download files from colab and upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Recurrent Neural Network

##useful for sequence data - use representation of previous input to aid the representation of a later input

In [83]:
### Model 2: LSTM
###LSTM = Long Short Term Memory (one of the most popular RNN)
'''
 input(text) -> toeknize -> embed -> layers (RNNs/Dense) -> Output
'''

'\n input(text) -> toeknize -> embed -> layers (RNNs/Dense) -> Output\n'

In [84]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x)
#print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [85]:
#Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [86]:
#compile
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [87]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220727-224220
Epoch 1/5
215/215 [==============================] - 4s 10ms/step - loss: 0.1071 - accuracy: 0.9657 - val_loss: 1.0187 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0421 - accuracy: 0.9796 - val_loss: 1.2154 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0382 - accuracy: 0.9800 - val_loss: 1.2526 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0402 - accuracy: 0.9804 - val_loss: 1.0269 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0431 - accuracy: 0.9783 - val_loss: 1.3069 - val_accuracy: 0.7690


In [88]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[3.4259376e-04],
       [8.9553624e-01],
       [9.9989188e-01],
       [3.3289699e-03],
       [9.9972039e-01],
       [2.0499539e-02],
       [8.8638262e-05],
       [9.9853566e-03],
       [9.9866128e-01],
       [9.9987066e-01]], dtype=float32)

In [89]:
#convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1.], dtype=float32)>

In [90]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)

In [91]:
model_2_results

{'accuracy': 76.9028871391076,
 'f1': 0.7681766761614921,
 'precision': 0.7680149813071524,
 'recall': 0.7690288713910761}

In [92]:
#Model 3: GRU
#GRU = Gated recurrent unit
##GRU is similar to LSTM, but has less parameters

In [93]:
from tensorflow.keras import layers

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")


inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

'''
# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x) 
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")
'''


'\n# Build an RNN using the GRU cell\ninputs = layers.Input(shape=(1,), dtype="string")\nx = text_vectorizer(inputs)\nx = model_3_embedding(x)\n# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True\nx = layers.GRU(64)(x) \n# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell\noutputs = layers.Dense(1, activation="sigmoid")(x)\nmodel_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")\n'

In [94]:
#compile
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [95]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [96]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20220727-224233
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.1150 - accuracy: 0.9587 - val_loss: 0.8455 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0495 - accuracy: 0.9791 - val_loss: 1.0266 - val_accuracy: 0.7493
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0429 - accuracy: 0.9793 - val_loss: 1.1630 - val_accuracy: 0.7402
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0374 - accuracy: 0.9816 - val_loss: 1.4144 - val_accuracy: 0.7441
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0361 - accuracy: 0.9820 - val_loss: 1.5737 - val_accuracy: 0.7467


In [97]:
model_3_preds = tf.squeeze(tf.round(model_3.predict(val_sentences)))
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1

In [98]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)

In [99]:
model_3_results

{'accuracy': 74.67191601049869,
 'f1': 0.7477031434071529,
 'precision': 0.7502930146455699,
 'recall': 0.7467191601049868}

In [100]:
#Model 4: Bidirectional RNN
##goes left to right and right to left

In [101]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4")

In [102]:
model_4.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-tra

In [103]:
#compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [104]:
# Fit model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Bidrectional")])

Saving TensorBoard log files to: model_logs/Bidrectional/20220727-224243
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.0860 - accuracy: 0.9717 - val_loss: 1.0973 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0410 - accuracy: 0.9783 - val_loss: 1.5554 - val_accuracy: 0.7559
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0372 - accuracy: 0.9799 - val_loss: 1.4531 - val_accuracy: 0.7415
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0358 - accuracy: 0.9825 - val_loss: 1.5713 - val_accuracy: 0.7362
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0329 - accuracy: 0.9829 - val_loss: 1.5450 - val_accuracy: 0.7362


In [105]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs

array([[1.84321223e-04],
       [9.57418382e-01],
       [9.99981403e-01],
       [4.83224588e-03],
       [9.99957442e-01],
       [1.14777058e-01],
       [1.16224401e-05],
       [3.22553678e-03],
       [9.99852896e-01],
       [9.99978662e-01],
       [2.14705055e-04],
       [2.67720083e-04],
       [9.99995947e-01],
       [1.40898395e-04],
       [3.57563258e-05],
       [7.83382449e-04],
       [9.99985933e-01],
       [4.57180664e-03],
       [9.97756183e-01],
       [9.86103415e-01],
       [6.49398714e-02],
       [2.30821432e-03],
       [9.99134958e-01],
       [9.76264477e-02],
       [7.09559827e-04],
       [9.99985576e-01],
       [6.71348185e-04],
       [9.37954392e-05],
       [9.99834061e-01],
       [9.99986887e-01],
       [9.99979973e-01],
       [4.44043719e-04],
       [1.12205325e-02],
       [5.72153585e-05],
       [5.78471518e-05],
       [9.99932766e-01],
       [9.27420388e-06],
       [1.50796380e-02],
       [6.38995171e-02],
       [1.24305938e-04],


In [106]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [107]:
model_4_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1.], dtype=float32)>

In [108]:
#calc the results of biredcational model
model_4_results = calculate_results(val_labels, model_4_preds)

In [109]:
model_4_results

{'accuracy': 73.62204724409449,
 'f1': 0.7370534084476977,
 'precision': 0.7388015784958836,
 'recall': 0.7362204724409449}

#Convolution Neural Networks for Text (and other sequences)

#Model 5: Conv1D NN

In [111]:
#Test out our embedding layer, Conv1d layer and pooling layer
embedding_test = embedding(text_vectorizer(["This is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # Get the most important feature
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [112]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0476912 ,  0.07679784, -0.01126306, ..., -0.07589082,
         -0.06985233, -0.01292253],
        [ 0.04254068,  0.01673366, -0.05724114, ...,  0.03838583,
         -0.06517895, -0.03987761],
        [-0.02990822, -0.05489602, -0.00608808, ...,  0.03421354,
          0.03112028, -0.01832725],
        ...,
        [ 0.04591256,  0.03087383,  0.00621633, ...,  0.00714362,
          0.00493412, -0.02824576],
        [ 0.04591256,  0.03087383,  0.00621633, ...,  0.00714362,
          0.00493412, -0.02824576],
        [ 0.04591256,  0.03087383,  0.00621633, ...,  0.00714362,
          0.00493412, -0.02824576]]], dtype=float32)>

In [113]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.14330956, 0.        , 0.12528147,
         0.        , 0.07458106, 0.03641946, 0.        , 0.        ,
         0.        , 0.0248801 , 0.        , 0.0381422 , 0.        ,
         0.        , 0.07845341, 0.        , 0.        , 0.08560348,
         0.        , 0.        , 0.00168963, 0.        , 0.03424046,
         0.00439279, 0.05277006, 0.        , 0.05605649, 0.        ,
         0.0953401 , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.03174412,
         0.        , 0.03170804, 0.01739686, 0.        , 0.00090978,
         0.00899437, 0.01627927, 0.13319923, 0.05416688, 0.01574154,
         0.        , 0.        , 0.02437087, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.01274404, 0.08225287,
         0.        , 0.05157555, 0.        , 0.        , 0.        ,
         0.03636798, 0.09527008],
        [0.07581975, 0.08713392, 0.        , 0.    

In [114]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.13807374, 0.14373095, 0.14330956, 0.        , 0.12602448,
        0.05944669, 0.07458106, 0.08220109, 0.03460443, 0.003852  ,
        0.00899437, 0.04555023, 0.13319923, 0.05416688, 0.04705513,
        0.11430027, 0.07845341, 0.02649251, 0.07903819, 0.08560348,
        0.04645869, 0.08604962, 0.01123144, 0.01274404, 0.08225287,
        0.00439279, 0.06683297, 0.0688541 , 0.05605649, 0.2002739 ,
        0.0953401 , 0.09527008]], dtype=float32)>

In [117]:
#Model 5: 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)


outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5")

In [120]:
model_5.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [121]:
model_5.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                 65  

In [122]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20220727-230433
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.1129 - accuracy: 0.9629 - val_loss: 1.0138 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0670 - accuracy: 0.9749 - val_loss: 1.1617 - val_accuracy: 0.7388
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0566 - accuracy: 0.9783 - val_loss: 1.2929 - val_accuracy: 0.7336
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0496 - accuracy: 0.9796 - val_loss: 1.3277 - val_accuracy: 0.7336
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0495 - accuracy: 0.9783 - val_loss: 1.3727 - val_accuracy: 0.7415


In [123]:
model_5_pred_probs = model_5.predict(val_sentences)

In [124]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [125]:
model_5_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1

In [126]:
model_5_results = calculate_results(val_labels, model_5_preds)

In [127]:
model_5_results

{'accuracy': 74.1469816272966,
 'f1': 0.7420648803393732,
 'precision': 0.7430922932503922,
 'recall': 0.7414698162729659}

#Universal Sentence Encoder transfer learning

In [131]:
#model 5
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,"When you call a USE on a sentence, it turn sit into numbers"])


In [132]:
sample_sentence

'A tornado destroyed my house'

In [133]:
embed_samples[0][:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([-0.01050084, -0.00229794, -0.05725909, -0.04777098,  0.04511795,
        0.02202453,  0.02400334,  0.0667739 , -0.00083695,  0.05448426,
        0.05745116, -0.04347977, -0.00139513,  0.01398681,  0.046919  ,
       -0.01260869, -0.06758504, -0.02162268, -0.00623832, -0.00575831,
        0.01941228, -0.00105522,  0.0304017 ,  0.06340312, -0.02500383,
       -0.07751907,  0.04895177, -0.04103375,  0.00698542, -0.0394792 ,
        0.03514631,  0.03659685, -0.05294777,  0.02191273, -0.02828331,
       -0.01387694,  0.05540748,  0.01839826, -0.02460213, -0.03052922,
        0.04902494, -0.0124156 , -0.01220245, -0.05936461, -0.08365225,
       -0.01479258, -0.0334257 , -0.00039739,  0.01519083, -0.01560132],
      dtype=float32)>

In [135]:
embed_samples[0].shape

TensorShape([512])

In [137]:
#Create keras layer using pretrained embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [138]:
#model 6
model_6 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(1, activation="sigmoid")
], name='model_6')

model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 1)                 513       
                                                                 
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [139]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Transfer_Learning_THUB")])

Saving TensorBoard log files to: model_logs/Transfer_Learning_THUB/20220727-232240
Epoch 1/5
215/215 [==============================] - 10s 28ms/step - loss: 0.6438 - accuracy: 0.7527 - val_loss: 0.6021 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.5789 - accuracy: 0.7856 - val_loss: 0.5488 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.5378 - accuracy: 0.7942 - val_loss: 0.5140 - val_accuracy: 0.7992
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.5105 - accuracy: 0.7957 - val_loss: 0.4895 - val_accuracy: 0.8005
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4916 - accuracy: 0.7984 - val_loss: 0.4719 - val_accuracy: 0.7992


In [140]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

In [141]:
model_6_results = calculate_results(val_labels,
                                    model_6_preds)

In [142]:
model_6_results

{'accuracy': 79.92125984251969,
 'f1': 0.7989121284115505,
 'precision': 0.7987486083787252,
 'recall': 0.7992125984251969}